# Лабораторная работа №2 Код Хемминга и контрольные суммы

Багильдинская Мария, БПМ-19-2

### Вариант №4, длина слова с контрольными битами = 36, алгоритм контрольной суммы: CRC-16

Т.к дана длина слова вместе с контольными битами, то определим сколько битов выделено для самого слова и контрольных бит:

Учитывая, что контрольные биты находяся только на позициях с номерами, равными степеням двойки, то контрольные биты будут находится на позициях 0, 1, 3, 7, 15 и 31 (с учетом индексации с нуля). Итого 6 контрольных бит, значит само слово занимает 30 бит.

In [84]:
positions = [0, 1, 3, 7, 15, 31]
lenw = 30

In [85]:
import pandas as pd
import random

#### Текст для передачи:

In [86]:
text = '''Модели обработки естественного языка (Natural language processing, NLP) на основе архитектуры Трансформеров, такие как BERT, RoBERTa, T5 или GPT3, успешно применяются в самых различных задачах и являются стандартом современных исследований в области NLP. Гибкость (универсальность) и надёжность Трансформеров способствовали их широкому распространению, что, в свою очередь, позволило легко адаптировать подобные модели для разнообразных задач обработки текстовых последовательностей, как в качестве seq2seq моделей для перевода, суммаризации, генерации текста и т.д., так и как самостоятельного энкодера для анализа тональности, частеречной разметки, машинного чтения и др. Главным изобретением Трансформеров стал механизм внутреннего внимания, который подсчитывает метрику схожести для всех возможных пар токенов входной последовательности независимо (параллельно), что позволяет избежать последовательной зависимости рекуррентных нейронных сетей. Благодаря этому механизму Трансформеры существенно превосходят более ранние модели обработки текстовых последовательностей, такие как LSTM.

Однако модели на основе Трансформеров и их производные имеют свои ограничения. Так, механизм внутреннего внимания требует вычислений, квадратичных длине входной последовательности. С нынешними мощностями и размерами моделей это обычно ограничивает входную последовательность до 512 токенов и не позволяет напрямую применять Трансформеры к задачам, требующим большего контекста, таким как вопросно-ответные системы, суммаризация документов или классификация фрагментов генома. Естественным образом возникает два вопроса:

1.	Можно ли достичь тех же эмпирических результатов, что и традиционные «квадратичные» Трансформеры, но используя разреженные модели, у которых требования к вычислительной мощности и памяти возрастают линейно с увеличением входной последовательности?
2.	Можно ли теоретически обосновать то, что эти «линейные» Трансформеры сохраняют выразительность и гибкость своих «квадратичных» собратов?

На оба эти вопроса авторы постарались ответить в своих недавних статьях. Так, в «ETC: Encoding Long and Structured Inputs in Transformers», представленной на конференции EMNLP 2020, было предложено построение расширенного Трансформера (Extended Transformer Construction, ETC) – новый метод разреженного внимания, в котором используется информация о структуре данных для ограничения числа подсчитываемых пар оценок сходства (similarity score). Это позволяет привести квадратичную зависимость к линейной и получить хорошие эмпирические результаты в NLP задачах.'''

print(text)

Модели обработки естественного языка (Natural language processing, NLP) на основе архитектуры Трансформеров, такие как BERT, RoBERTa, T5 или GPT3, успешно применяются в самых различных задачах и являются стандартом современных исследований в области NLP. Гибкость (универсальность) и надёжность Трансформеров способствовали их широкому распространению, что, в свою очередь, позволило легко адаптировать подобные модели для разнообразных задач обработки текстовых последовательностей, как в качестве seq2seq моделей для перевода, суммаризации, генерации текста и т.д., так и как самостоятельного энкодера для анализа тональности, частеречной разметки, машинного чтения и др. Главным изобретением Трансформеров стал механизм внутреннего внимания, который подсчитывает метрику схожести для всех возможных пар токенов входной последовательности независимо (параллельно), что позволяет избежать последовательной зависимости рекуррентных нейронных сетей. Благодаря этому механизму Трансформеры существенно 

Всего в тексте 90 уникальных символов:

In [87]:
len(set(list(text)))

90

Поэтому закодируем 1 символ 7 битами. Сопоставление между символами и двоичным кодом представлено ниже.

In [88]:
bin_code = pd.DataFrame(data = set(list(text)), columns = ['char'])
bin_code['binary_code'] = list('{0:07b}'.format(x) for x in range(1, 91))
bin_code.head(5)

,char,binary_code
0,ф,0000001
1,m,0000010
2,щ,0000011
3,Г,0000100
4,»,0000101


Напишем функцию кодирования текста кодом Хемминга, одновременно считая контрольные суммы CRC-16. Для CRC-16 полиномом будет , то есть 11000000000000101. Контрольная сумма будет записана в последних 16 битах. 

Функция divide - для вычисления CRC16 по прямому алгоритму

In [89]:
def toList(x):
  l = []
  for i in range (0,len(x)):
    l.append(int(x[i]))
  return (l)

def toString(x):
  str1 =""
  for i in range (0,len(x)):
    str1+=str(x[i])
  return (str1)

def divide(val1,val2 = '11000000000000101'):
  a = [int(i) for i in list(str(int(val1)))]
  b = [int(i) for i in val2]
  while len(b) <= len(a):
    del a[0]
    for j in range(len(b)-1):
      a[j] ^= b[j+1]
    a = [int(i) for i in list(str(int(toString(a))))]
  return toString(a)

In [90]:
def encode_HammingCRC16(text):
    #переход к бинарному представлению текста
    txt_toBinary = (bin_code[bin_code.char == x]['binary_code'].values[0] for x in list(text))
    binStr = ''.join(list(txt_toBinary))

    #разделение на 30-битные информационные слова
    bin_package30 = [binStr[i:i+lenw] for i in range(0, len(binStr), lenw)]

    #дополнение нулями последнего слова
    if len(bin_package30[-1]) != lenw: bin_package30[-1] += (lenw - len(bin_package30[-1]))*'0'
    #добавление контрольных бит и контрольных сумм CRC16
    DataBits = []
    for i in bin_package30:
        tmp = list(i)
        crc = divide(str(int(i + 16*'0')))
        if len(crc) != 16:
            crc = (16 - len(crc))*'0' + crc
        for j in positions:
            tmp.insert(j, '0')
        for j in positions:
            sm = 0
            for k in range(j, len(tmp), 2*(j+1)):
                sm += sum(int(h) for h in tmp[k:k+j+1])
            tmp[j] = str(sm%2)
        DataBits.append(''.join(tmp) + crc)
        
    return ''.join(DataBits)

encode_HammingCRC16(text)

'...'

При добавлении ошибок случайно выбираются 100 информационных слов. Функция добавления ошибок:

In [91]:
def include_errors(binStr, n_errors):
    #разделение на 52-битные отрезки (30 - информационное слово, 6 - контрольные биты, 16 - контрольная сумма CRC)
    n = 52
    bin_package52 = [binStr[i:i+n] for i in range(0, len(binStr), n)]
    #добавление ошибок
    ErrorsBits = []
    chosen_words = []

    all_words = list(range(len(bin_package52)))

    for i in range(100):
        chosen_words.append(all_words.pop(all_words.index(random.choice(all_words))))

    for i in range(len(bin_package52)):
        if i in chosen_words:
            tmp = list(bin_package52[i])
            all_chars = [2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]
            all_chars.extend(list(range(16, 31)))
            all_chars.extend(list(range(32, 36)))
            
            chosen_chars = []
            for j in range(n_errors):
                chosen_chars.append(all_chars.pop(all_chars.index(random.choice(all_chars))))
            
            for j in chosen_chars:
                tmp[j] = str(int(tmp[j])^1)
            ErrorsBits.append(''.join(tmp))
        else: ErrorsBits.append(bin_package52[i])
    return ''.join(ErrorsBits)

Функция декодирования с исправлением ошибок:

In [92]:
def decode_HammingCRC(binStr):
    #разделение на 52-битные отрезки
    n = 52
    bin_package52 = [binStr[i:i+n] for i in range(0, len(binStr), n)]
    #разделение на информационное слово с контрольными битами и CRC
    for i in range(len(bin_package52)):
        bin_package52[i] = [bin_package52[i][:36], bin_package52[i][36:]]
    
    DataBits30 = []

    for i in bin_package52:
        tmp = list(i[0])
        #извлечение контрольных бит
        for j in [31, 15, 7, 3, 1, 0]:
            tmp.pop(j)
        DataBits30.append(''.join(tmp))

    DataBits = []

    wrongWords_found = 0
    fixed = 0
    crc_wr = 0
    er_fixed = 0
    for i in range(len(DataBits30)):
        #подсчет контрольных бит и CRC
        tmp = list(DataBits30[i])

        crc = divide(DataBits30[i] + 16*'0')
        if len(crc) != 16:
            crc = (16 - len(crc))*'0' + crc

        if crc != bin_package52[i][1]:
            crc_wr += 1

        for j in positions:
            tmp.insert(j, '0')
        for j in positions:
            sm = 0
            for k in range(j, len(tmp), 2*(j+1)):
                sm += sum(int(h) for h in tmp[k:k+j+1])
            tmp[j] = str(sm%2)
        #сравнение с исходными контрольными битами и crc
        sm = 0
        for j in positions:
            if bin_package52[i][0][j] != tmp[j]:
                sm += j + 1
            #извлечение контрольных бит
        
        if sm != 0:

            wrongWords_found += 1
            #исправление ошибки
            if len(tmp) >= sm:
                er_fixed += 1
                tmp[sm-1] = str(int(tmp[sm-1])^1)

            for j in [31, 15, 7, 3, 1, 0]:
                tmp.pop(j)

            #повторный подсчет crc
            crc = divide(''.join(tmp) + 16*'0')
            if len(crc) != 16:
                crc = (16 - len(crc))*'0' + crc

            if crc == bin_package52[i][1]:
                fixed += 1
                for s in tmp:
                    DataBits.append([s, 0])
            else:
                for s in tmp:
                    DataBits.append([s, 1])
        else:
            for j in [31, 15, 7, 3, 1, 0]:
                tmp.pop(j)
                
            if crc != bin_package52[i][1]:
                wrongWords_found += 1
                for s in tmp:
                    DataBits.append([s, 1])
            else:
                for s in tmp:
                    DataBits.append([s, 0])
    i = 0
    fin = []

    while len(DataBits) > 7:
        sm = 0
        tmp = []

        for i in range(7):
            sm += DataBits[i][1]
            tmp.append(DataBits[i][0])
        if sm == 0: fin.append(''.join(tmp))
        DataBits = DataBits[7:]
    for i in range(len(fin)-1, len(fin)-5, -1):
        if fin[i] == "0"*7: fin.pop(i)
    BinarytoText = [bin_code[bin_code.binary_code == x]['char'].values[0] for x in fin]

    return ''.join(BinarytoText), wrongWords_found, fixed, len(bin_package52), crc_wr, er_fixed

### Отправка без ошибок:

In [93]:
txt, wr, fixed, nw, crc_wr, er_fixed = decode_HammingCRC(encode_HammingCRC16(text)) 
print(txt)
print()
print("Количество неверно переданных информационных слов:", wr)
print("Количество исправленных ошибок:", fixed)
print("Количество инф.слов всего:", nw)
print("Количество инф.слов с неверным crc:", crc_wr)
print("Количество исправленных битов:", er_fixed)

Модели обработки естественного языка (Natural language processing, NLP) на основе архитектуры Трансформеров, такие как BERT, RoBERTa, T5 или GPT3, успешно применяются в самых различных задачах и являются стандартом современных исследований в области NLP. Гибкость (универсальность) и надёжность Трансформеров способствовали их широкому распространению, что, в свою очередь, позволило легко адаптировать подобные модели для разнообразных задач обработки текстовых последовательностей, как в качестве seq2seq моделей для перевода, суммаризации, генерации текста и т.д., так и как самостоятельного энкодера для анализа тональности, частеречной разметки, машинного чтения и др. Главным изобретением Трансформеров стал механизм внутреннего внимания, который подсчитывает метрику схожести для всех возможных пар токенов входной последовательности независимо (параллельно), что позволяет избежать последовательной зависимости рекуррентных нейронных сетей. Благодаря этому механизму Трансформеры существенно 

In [94]:
txt, wr, fixed, nw, crc_wr, er_fixed = decode_HammingCRC(include_errors(encode_HammingCRC16(text), 1)) 
print(txt)
print()
print("Количество неверно переданных информационных слов:", wr)
print("Количество исправленных ошибок:", fixed)
print("Количество инф.слов всего:", nw)
print("Количество инф.слов с неверным crc:", crc_wr)
print("Количество исправленных битов:", er_fixed)

Модели обработки естественного языка (Natural language processing, NLP) на основе архитектуры Трансформеров, такие как BERT, RoBERTa, T5 или GPT3, успешно применяются в самых различных задачах и являются стандартом современных исследований в области NLP. Гибкость (универсальность) и надёжность Трансформеров способствовали их широкому распространению, что, в свою очередь, позволило легко адаптировать подобные модели для разнообразных задач обработки текстовых последовательностей, как в качестве seq2seq моделей для перевода, суммаризации, генерации текста и т.д., так и как самостоятельного энкодера для анализа тональности, частеречной разметки, машинного чтения и др. Главным изобретением Трансформеров стал механизм внутреннего внимания, который подсчитывает метрику схожести для всех возможных пар токенов входной последовательности независимо (параллельно), что позволяет избежать последовательной зависимости рекуррентных нейронных сетей. Благодаря этому механизму Трансформеры существенно 

Символы, содержащие биты из неверно переданного информационного слова, пропускаются

In [95]:
txt, wr, fixed, nw, crc_wr, er_fixed = decode_HammingCRC(include_errors(encode_HammingCRC16(text), 12)) 
print(txt)
print()
print("Количество неверно переданных информационных слов:", wr)
print("Количество исправленных ошибок:", fixed)
print("Количество инф.слов всего:", nw)
print("Количество инф.слов с неверным crc:", crc_wr)
print("Количество исправленных битов:", er_fixed)

Модели обрабстественного языка (Natural language proce, Nа основетектуры Трансформеров, такие как BERT, RoBERTa, T5 или  успешно применя самых разлизадачахяютандартомеменных исследований в области NLP. Гибкость (унивеность) и надёжность Трансформеров способствовали их широкому распро что, в оче позволигко адапподобные модели для разнообраадач обработки текстовых последователей, как в качестве seq2seq моддлявода, суммари, генерации текста и т.д., так и как самостоятельного энкодера для анализа тональносастеречнзметки, ного чтения и др. Главным изобретением Трансформеров стал механизм внутреннего вня, который подсчет метрику сти для всех возможных пар токенов входной послеельности независимо (парано), чтооляет избежать послеельной завис рекуррентных нейронных сетей. Благодаря этому механизму Трансформеры существенно превосходят более ранние модели ткитовых повательностейие кTM.

Однако модели на оснрансформеров производные имеои ограничения. Так, механизм внутго внимания ет вычислений, к